In [1]:
import os, glob
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [2]:
from totalface_cpu.model_zoo.get_models import get_detection_model,get_recognition_model
from totalface_cpu.face.get_result import get_detection,get_features
from totalface_cpu.data import read_image
from totalface_cpu.utils.util_common import draw_result

/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.23ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pkg_resources/_vendor/packaging/version.py:111: DeprecationWarning: Creating a LegacyVersion has been deprecated and will be removed in the next major release
  warnings.warn(


In [3]:
# detection
detection_name = "scrfd"
detection_path = "scrfd_10g_bnkps.onnx"

# Set parameters
detection_thresh = 0.5
detection_height_min=0 

# if load multiple trt, load_multi=True, default False
detection_model = get_detection_model(detection_name,detection_path,load_multi=False)

providers: ['CPUExecutionProvider']
scrfd onnx loaded


In [4]:
# wild

In [5]:
img_prefix = "/data/"

In [6]:
base = "FaceInTheWild/{}/image_list_org_1_80x60_new_unmask.txt"

In [7]:
tv = ['Training','Validation']

In [8]:
txt_path = base.format(tv[1])

In [9]:
lines = open(txt_path,'r').readlines()

In [24]:
for li,line in enumerate(lines):
    sp = line.strip().split(",")
    ori_sub,new_sub,_,_,id_val,age,gender,_,_,lb_str = sp
    
    ori_path = os.path.join(img_prefix,ori_sub)
    img_rgb = read_image(ori_path)
    
    faces = get_detection(detection_name,detection_model,img_rgb,thresh=detection_thresh,height_min=detection_height_min,input_size=(640,640))
    
    if len(faces)==1:
        face = faces[0]
    elif len(faces)<1:
        print("not detect")
    else:
        for fc in faces:
            if fc['max_flag']:
                face = fc
                break
                
    box = face['bbox'].astype(np.int32)
    box = np.maximum(box, 0)
    
    cropped = img_rgb[int(box[1]):int(box[3]), int(box[0]):int(box[2])]
    cropped = cv2.resize(cropped,(224,224))
    cropped_bgr = cv2.cvtColor(cropped,cv2.COLOR_RGB2BGR)
    break
    
    

In [25]:
new_name = "{}_{}.jpg".format(li,id_val)

In [29]:
base = "/data/shared/Face/FaceLiveness/datasets/casia-fasd_images/original/{}_img/color/"
tv=['train','test']

In [32]:
img_list = glob.glob(base.format(tv[1])+"*.jpg")+glob.glob(base.format(tv[1])+"*.png")

In [36]:
img_dict={}
for imgpath in img_list:
    img_name = imgpath.split("/")[-1]
    id_val = img_name.split(".avi")[0]
    if not id_val in img_dict:
        img_dict[id_val]=[]
    img_dict[id_val].append(imgpath)

In [50]:
p1 = img_dict['30_7'][0]
p2 = img_dict['30_7'][5]
img1 = read_image(p1)
img2 = read_image(p2)